## <b><font color='darkblue'>Preface</font></b>
([course source](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/3/langchain-expression-language-(lcel))) <b><font size='3ptx'>LCEL (LangChain Expression Language)</font></b>

LCEL is a declarative language within LangChain for easily composing chains of components. It offers a simpler way to define and execute complex workflows involving LLMs, data loading, and other processing steps.

If you're building applications with LangChain, LCEL can be a valuable tool for streamlining your development process.

In [1]:
!pip freeze | grep -P '(openai|langchain)'

langchain==0.2.6
langchain-anthropic==0.1.15
langchain-community==0.2.6
langchain-core==0.2.10
langchain-experimental==0.0.62
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
openai==1.28.1


In [5]:
import json
import os
import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv

import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## <b><font color='darkblue'>Simple Chain</font></b>

In [6]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

In [7]:
chain = prompt | model | output_parser

In [8]:
resp = chain.invoke({"topic": "bears"})

In [9]:
resp

"Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship anymore!"

## <b><font color='darkblue'>More complex chain</font></b>
And Runnable Map to supply user-provided inputs to the prompt.

In [12]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [13]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [15]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [16]:
retriever.invoke("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [18]:
from langchain.schema.runnable import RunnableMap

In [19]:
chain = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [20]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [23]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

In [24]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## <b><font color='darkblue'>Bind</font></b>
Sometimes we want to invoke a Runnable within a Runnable sequence with constant arguments that are not part of the output of the preceding Runnable in the sequence, and which are not part of the user input. We can use Runnable.bind() to pass these arguments in. ([more](https://python.langchain.com/v0.1/docs/expression_language/primitives/binding/))

In [26]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }]

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [28]:
runnable = prompt | model

In [29]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e0724e52-3506-4f16-808b-70a91e7100ad-0', usage_metadata={'input_tokens': 64, 'output_tokens': 16, 'total_tokens': 80})

In [30]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }, {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }]

In [31]:
model = model.bind(functions=functions)

In [32]:
runnable = prompt | model

In [33]:
resp = runnable.invoke({"input": "how did the patriots do yesterday?"})

In [34]:
resp

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0bafb4a6-53a0-4e52-be72-2cf0070d2e62-0', usage_metadata={'input_tokens': 99, 'output_tokens': 18, 'total_tokens': 117})

## <b><font color='darkblue'>Fallbacks</font></b>
Due to the deprecation of OpenAI's model text-davinci-001 on 4 January 2024, you'll be using OpenAI's recommended replacement model gpt-3.5-turbo-instruct instead.

In [35]:
from langchain.llms import OpenAI
import json

In [36]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [37]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [38]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

The next line is expected to fail.

In [40]:
# JSONDecodeError: Extra data: line 9 column 1 (char 125)
# simple_chain.invoke(challenge)

In [41]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [42]:
chain.invoke(challenge)

{'poem1': {'title': 'The Night Sky',
  'author': 'Emily Dickinson',
  'firstLine': 'The night is starry and the stars are blue.'},
 'poem2': {'title': 'Autumn Leaves',
  'author': 'Robert Frost',
  'firstLine': "My sorrow, when she's here with me, thinks these dark days of autumn rain are beautiful as days can be."},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul.'}}

In [43]:
final_chain = simple_chain.with_fallbacks([chain])

In [44]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Night Sky',
  'author': 'Emily Dickinson',
  'firstLine': 'The night is starry and the stars are blue.'},
 'poem2': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'},
 'poem3': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood,'}}

## <b><font color='darkblue'>Interface</font></b>

In [45]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [46]:
chain.invoke({"topic": "bears"})

'Why did the bear bring a flashlight to the party? \nBecause he wanted to be the "light" of the party!'

In [47]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why did the bear dissolve in water?\n\nBecause it was polar!',
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

In [48]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?
 


Because
 they
 have
 bear
 feet
!



In [49]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears like fast food? Because they can't catch it!"

## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearning.ai - Functions, Tools and Agents with LangChain Ch4: OpenAI Function Calling In LangChain](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/4/openai-function-calling-in-langchain)